## Setup

Add the project root and import the loss classes used in this notebook.


In [ ]:
import sys
sys.path.insert(0, "../../../src")

import numpy as np
from didgelab import (
    CompositeTairuaLoss,
    FrequencyTuningLoss,
    ScaleTuningLoss,
    PeakQuantityLoss,
    PeakAmplitudeLoss,
    QFactorLoss,
    ModalDensityLoss,
    IntegerHarmonicLoss,
    NearIntegerLoss,
    StretchedOddLoss,
    HighInharmonicLoss,
    HarmonicSplittingLoss,
    note_to_freq,
)


# Full example: evolution with composite loss

Combine several components and run a short evolution. Requires `Nuevolution`, a genome (e.g. `GeoGenomeA`), and optionally `init_standard_evolution` for monitoring.

In [ ]:
from didgelab import Nuevolution, GeoGenomeA, init_standard_evolution
from didgelab.app import init_app

init_app("loss_example", create_output_folder=False)

target_freqs_hz = np.array([73.4, 146.8])
target_freqs_log = np.log2(target_freqs_hz)
target_impedances = np.array([-1.0, -1.0])  # -1 = frequency-only; use [0,1] to target amplitudes

loss = CompositeTairuaLoss(max_error=5.0)
loss.add_component("freq", FrequencyTuningLoss(target_freqs_log, target_impedances, [1.0, 1.0]))
loss.add_component("scale", ScaleTuningLoss(60, [0, 2, 4, 5, 7, 9, 11], weight=5.0))
loss.add_component("peaks", PeakQuantityLoss(target_count=4, weight=2.0))
loss.add_component("volume", PeakAmplitudeLoss(0.2, weight=10.0))
loss.target_freqs = target_freqs_hz

evo = Nuevolution(
    loss,
    GeoGenomeA.build(5),
    num_generations=4,
    population_size=6,
    generation_size=4,
)
init_standard_evolution(loss.target_freqs, evo)
population = evo.evolve()
print("Best loss:", population[0].loss)